In [2]:
# https://www.kaggle.com/shivamkushwaha/bbc-full-text-document-classification
!wget -nc https://lazyprogrammer.me/course_files/nlp/bbc_text_cls.csv

--2024-03-09 04:05:42--  https://lazyprogrammer.me/course_files/nlp/bbc_text_cls.csv
Resolving lazyprogrammer.me (lazyprogrammer.me)... 172.67.213.166, 104.21.23.210, 2606:4700:3031::6815:17d2, ...
Connecting to lazyprogrammer.me (lazyprogrammer.me)|172.67.213.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5085081 (4.8M) [text/csv]
Saving to: ‘bbc_text_cls.csv’

bbc_text_cls.csv    100%[===================>]   4.85M  --.-KB/s    in 0.05s   

2024-03-09 04:05:43 (99.6 MB/s) - ‘bbc_text_cls.csv’ saved [5085081/5085081]



In [3]:
import pandas as pd
import numpy as np
import nltk

from nltk import word_tokenize

In [4]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
df = pd.read_csv('bbc_text_cls.csv')

In [6]:
df.head()

,text,labels
0,Ad sales boost Time Warner profit\n\nQuarterly...,business
1,Dollar gains on Greenspan speech\n\nThe dollar...,business
2,Yukos unit buyer faces loan claim\n\nThe owner...,business
3,High fuel prices hit BA's profits\n\nBritish A...,business
4,Pernod takeover talk lifts Domecq\n\nShares in...,business


In [7]:
#populate word2idx
#convert documents into sequence of ints / ids / indices
idx = 0
word2idx = {}
tokenized_docs = []
for doc in df['text']:
  words = word_tokenize(doc.lower())
  doc_as_int = []
  for word in words:
    if word not in word2idx:
      word2idx[word] = idx
      idx += 1

# save for later
doc_as_int.append(word2idx[word])
tokenized_docs.append(doc_as_int)

Bước tiếp theo là đưa từ của chúng ta vào ánh xạ tiếp theo.

Điều này là cần thiết để chúng ta biết từ nào ánh xạ tới cột nào trong ma trận TFIDF sẽ bắt đầu bằng cách khởi tạo chỉ số về 0.

Chúng tôi cũng sẽ tạo một danh sách trống nơi chúng tôi sẽ lưu trữ các tài liệu được mã hóa của mình.

Bước tiếp theo là lặp qua cột văn bản của khung dữ liệu bên trong vòng lặp này.

Chúng tôi bắt đầu bằng cách viết thường và mã hóa tài liệu hiện tại mà chúng tôi gọi là các từ kết quả.


Bước tiếp theo là tạo một danh sách trống có tên doc_as_int.

Bước tiếp theo là Chúng tôi kiểm tra xem từ hiện tại có trong IDX hay không.

Nếu không có thì nghĩa là từ này nó chưa có chỉ mục tương ứng.

Vì vậy, nếu đúng như vậy, chúng tôi sẽ thêm từ này vào từ điển của mình trong đó khóa là từ có giá trị là index.

Bước tiếp theo là tăng chỉ số lên một để nó có giá trị mới vào lần sử dụng tiếp theo.

Tại thời điểm này, chúng tôi biết rằng từ của chúng tôi có index tương ứng vì chúng tôi vừa tạo nó hoặc nó đã tồn tại rồi.

Do đó, chúng ta có thể lấy chỉ mục của nó từ word sang IDX và sau đó lưu nó vào danh sách của mình.


In [8]:
# reverse mapping
# if you do it smarter you can store it as a list
idx2word = {v:k for k, v in word2idx.items()}

Bước tiếp theo là tạo ánh xạ ngược.

Vì vậy, mã này lặp qua từ điển từ để lập index của chúng tôi và chỉ đơn giản biến giá trị thành khóa và khóa, giá trị.

Lưu ý rằng điều này hơi kém hiệu quả vì chúng ta đang sử dụng cấu trúc dữ liệu từ điển, mặc dù chỉ số của chúng tôi chỉ là số nguyên từ 0 cho đến kích thước từ vựng.

Thay vào đó, sẽ hiệu quả hơn nếu lưu trữ một danh sách.


In [9]:
#number of documents
N = len(df['text'])

In [10]:
#number of words
V = len(word2idx)

In [11]:
# instantiate term_frequency matrix
#note: could have also used count vectorizer
tf = np.zeros((N,V))

Bước tiếp theo là khởi tạo ma trận tần số thuật ngữ của chúng ta.

Lưu ý rằng đây giống như ma trận vectơ đếm của chúng ta.

Bây giờ, trên thực tế, mặc dù sẽ hiệu quả hơn nếu sử dụng ma trận thưa thớt để khiến mọi thứ trở nên hơi rắc rối, phức tạp hơn mà không cải thiện sự hiểu biết của bạn.

Vì vậy, trong tập lệnh này, chúng ta sẽ sử dụng một ma trận dày đặc, đây chỉ là một mảng Nampai thông thường.

Sử dụng ma trận thưa sẽ là bài tập để mở rộng những gì bạn đã học trong bài giảng này.

OK, như bạn có thể thấy, chúng ta khởi tạo ma trận này là ma trận các số 0 có kích thước n x V.

In [12]:
#populate term-frequency counts
for i, doc_as_int in enumerate(tokenized_docs):
  for j in doc_as_int:
    tf[i,j] += 1

Bước tiếp theo là điền vào ma trận tần số thuật ngữ của chúng tôi, để thực hiện việc này, chúng tôi chỉ cần lặp qua tokenized documents của mình.

Như bạn nhớ lại, chúng tôi đã lưu trữ tất cả chúng dưới dạng danh sách ID số nguyên.

Lưu ý rằng chúng tôi sử dụng Enumerate, nó cung cấp cho chúng tôi chỉ mục cho tài liệu cùng một lúc, chúng tôi sẽ ký hiệu bằng chữ I.

Vì vậy, nó cho chúng ta biết tài liệu nào chúng ta đang xem bên trong tài liệu đầu tiên.

Chúng tôi có vòng lặp thứ hai đi qua từng index từ trong tài liệu.

Vì đây đã là các chỉ số từ nên chúng ta sẽ gọi chúng là J.

Vì vậy, để điền vào ma trận tần số thuật ngữ của chúng tôi. Chúng ta chỉ cần tăng giá trị ở vị trí I.J lên một. Điều này có nghĩa là trong văn bản I, chúng ta đã thấy từ thứ i một lần nữa.



In [13]:
# compute IDF
document_freq = np.sum(tf > 0, axis = 0) #document frequency (shape = (V,))
idf = np.log(N / document_freq)

<ipython-input-13-d39c34d5fbff>:3: RuntimeWarning: divide by zero encountered in divide
  idf = np.log(N / document_freq)


In [14]:
#compute TF-IDF
tf_idf = tf * idf

<ipython-input-14-8bdbac542b36>:2: RuntimeWarning: invalid value encountered in multiply
  tf_idf = tf * idf


In [15]:
np.random.seed(123)

In [16]:
# pick a random document, show the top 5 terms ( in terms of tf_idf score)
i = np.random.choice(N)
row = df.iloc[i]
print("Label:", row['labels'])
print("Text:", row['text'].split("\n",1)[0])
print("Top 5 terms:")

scores = tf_idf[i]
indices = (-scores).argsort()

for j in indices[:5]:
  print(idx2word[j])

Label: sport
Text: Athens memories soar above lows
Top 5 terms:
lol
murderers
sentencers
denham
aggravating
